# Llamaindex and Typesense

### Notes to self on getting the right version of Python installed on my new Win machine as well as setting up VS Code to use the correct version

List the installed versions of Python
> py --list

Install a specific version of Python
> winget install Python.Python.3.10

Run a specific version of Python
> py -3.10

List the paths to set VS Code to use the correct Python version
> py -0p

## Instructions

https://docs.llamaindex.ai/en/stable/examples/vector_stores/TypesenseDemo/

In [ ]:
!pip install llama-index-embeddings-openai
!pip install llama-index-vector-stores-typesense

In [ ]:
!pip install python-dotenv

In [ ]:
!pip install llama-index

In [14]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
)
from IPython.display import Markdown, display

from dotenv import load_dotenv
import textwrap
import os

load_dotenv()  # This loads the variables from .env
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")


In [5]:
# load documents
documents = SimpleDirectoryReader("./data/john_maeda/").load_data()

In [15]:
from llama_index.vector_stores.typesense import TypesenseVectorStore
from typesense import Client

# Connect to Typesense Cloud
client = Client({
    'api_key': os.getenv('TYPESENSE_API_KEY'),
    'nodes': [{
        'host': os.getenv('TYPESENSE_HOST'),
        'port': os.getenv('TYPESENSE_PORT'),
        'protocol': os.getenv('TYPESENSE_PROTOCOL')
    }]
})
collection_name = "maeda_collection"
typesense_vector_store = TypesenseVectorStore(typesense_client, collection_name=collection_name)
storage_context = StorageContext.from_defaults(
    vector_store=typesense_vector_store
)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context
)

###  Using vector search

In [ ]:
from llama_index.core import QueryBundle
from llama_index.embeddings.openai import OpenAIEmbedding

# By default, typesense vector store uses vector search. You need to provide the embedding yourself.
query_str = "What is the relationship between art and technology?"
embed_model = OpenAIEmbedding()
# You can also get the settings from the Settings object
from llama_index.core import Settings

# embed_model = Settings.embed_model
query_embedding = embed_model.get_agg_embedding_from_queries(query_str)
query_bundle = QueryBundle(query_str, embedding=query_embedding)
response = index.as_query_engine().query(query_bundle)

display(Markdown(f"{response}"))

### Using text search

In [ ]:
from llama_index.core.vector_stores.types import VectorStoreQueryMode

# You can also use text search
query_str = "What is the relationship between art and technology?"
query_str = "What is the STEAM?"

query_bundle = QueryBundle(query_str=query_str)
response = index.as_query_engine(
    vector_store_query_mode=VectorStoreQueryMode.TEXT_SEARCH
).query(query_bundle)
display(Markdown(f"{response}"))

In [ ]:
response = typesense_client.collections[collection_name].delete()
print(response)